In [ ]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

import os

import data as dat
from model import Model

import colours as c

dir = os.getcwd()
dir_main = os.path.dirname(os.path.abspath(dir))

In [ ]:
scorer = dat.Validator()
m = Model(scorer.smiles_vocabulary, dropout=False)

In [ ]:
with tf.Session() as session:
    
    saver = tf.train.Saver()
    saver.restore(session, dir_main + "/trained/Solanaceae-L")
    
    print("Model restored.")
    
    # Score the data iteratively
    smiles, labels, weights, smiles_raw = scorer.getEvaluations()
    
    correct = [0] * scorer.classes_amount
    total = [0] * scorer.classes_amount
    
    for smile, label, weight, smile_raw in zip(smiles, labels, weights, smiles_raw):
        
        output = session.run([m.output], feed_dict={m.batch_size: 1, m.input: np.expand_dims(smile, 0)})
        output = list(np.round(output)[0,0])
        
        for i in range(len(output)):
            
            if label[i] != 0.5:
            
                if output[i] == label[i]:
                    correct[i] += 1

                total[i] += 1
                
        print("\nClasses: {}".format(scorer.classes))
        print("Correct: {}".format(correct))
        print("Total: {}".format(total))
        
        percentages = list(np.array(correct)/np.array(total) * 100)
        print("Percentages: {}".format(percentages))
        
    figure = plt.figure(figsize=(20, 5))
    plt.bar(scorer.classes, percentages, color="blue")